# LunAPI : advanced notes

Links to notebooks in this repository: [Index](./00_overview.ipynb) | [Individuals](./01_indivs.ipynb) | [Projects](./02_projects.ipynb) | [Staging](./03_staging.ipynb) | [Models](./04_models.ipynb) | [Advanced](./98_advanced.ipynb) | [Reference](./99_reference.ipynb)

In [6]:
import lunapi as lp
proj = lp.proj()

initiated lunapi v0.0.4 <lunapi.lunapi0.luna object at 0xffff6c1a3cf0> 

enabling console outputs


## Structure of the `lunapi` package


[lunapi/lunapi1.py](https://github.com/remnrem/luna-api/blob/c592e23fe3ecec03774ed245062cab797aee200c/src/lunapi/lunapi1.py)

[lunapi/lunapi0.cpp](https://github.com/remnrem/luna-api/blob/c592e23fe3ecec03774ed245062cab797aee200c/src/lunapi/lunapi0.cpp)

### lunapi1

### lunapi0

## Luna variables

In [ ]:
correspondence w/ luna command line

## Controlling inputs

In [10]:
### Setting Luna variables

In [ ]:
### Multi-line Luna commands

In [10]:
p.eval( """ MASK ifnot=N2
            RE
            PSD sig=EEG spectrum dB """ )

 ..................................................................
 CMD #1: MASK
   options: ifnot=N2 sig=*
  set masking mode to 'force'
  annots: N2
  applied annotation mask for 1 annotation(s)
  399 epochs match; 796 newly masked, 0 unmasked, 399 unchanged
  total of 399 of 1195 retained
 ..................................................................
 CMD #2: RE
   options: sig=*
  restructuring as an EDF+:  retaining 11970 of 35850 records
  of 597.5 minutes, dropping 398, retaining 199.5
  resetting mask
  clearing any cached values and recording options
  retaining 399 epochs
 ..................................................................
 CMD #3: PSD
   options: dB sig=EEG spectrum
  calculating PSD from 0.5 to 25 for 1 signals


,Command,Stata
0,MASK,EMASK
1,PSD,B_CH
2,PSD,CH
3,PSD,CH_F
4,RE,BL


### Parsing command line files

### Running command files

In [ ]:
lp.cmdfile( 'tutorial/cmd/first.txt')

In [ ]:
proj.opt( 'stage' , 'N2' )
p.eval( lp.cmdfile( 'tutorial/cmd/first.txt') )b

In [ ]:
p.strata()

In [ ]:
p.table( 'STATS' , 'CH_STAGE' )

### Import external databases

In [ ]:
proj.import_db( 'out.db' ) 

In [ ]:
proj.commands()

In [ ]:
proj.table( 'HEADERS' )

## Controlling output

### Silencing log output

### Writing to text-tables

## Instance referencing

When creating an individual instance, __the returned object is only a reference to the instance, not the instance itself__.  This distinction can be important to note when working with multiple instances, or trying to copy instances, etc.   In brief, if we start with this: 

In [13]:
p = proj.inst('id1')

In [15]:
q = p

At this point, the following statements hold:

 - `q` points to the same object as `p`
 - therefore, if `p` is changed, so is `q` (and vice versa)
 - if `p` is deleted (`del p`), `q` will continue to point to the same Luna instance `id1` however
 - if both `p` and `q` are deleted, then Python's garbage collector will release any resources used to store `id1` (i.e. based on Python's _reference-counting_ approach to memory management - meaning that you normally shouldn't have to worry about explicitly freeing resources

This of course begs the question: if you _do_ want to make a copy of an instance, how is that done?   This is done by 

<img src="img/py-refs.png" width=40% height=40%>

# Advanced topics

This section briefly covers a few more advanced topics to help orient working with `lunapi`

## `lunapi1` and `lunapi0` functions

In the example below, `edf` is the level-1 object (i.e. wrapper plus instance), whereas `raw` is the level-0 object (i.e. a vanilla instance).   Most functions are similar - it is just the output format that is different (e.g. a pandas dataframe versus a list-of-lists, etc).

In [ ]:
edf

In [ ]:
edf.channels()

Here we get the raw, level-0 instance (the `edf` member of the level-1 instance):

In [ ]:
raw = edf.edf

In [ ]:
raw.channels()

In [ ]:
raw

As a second example, using `stat()`:

In [ ]:
edf.stat()

In [ ]:
raw.stat()

## `lunapi-inst` objects are references

When a new instance is created, this happens _internally_ within the Luna C/C++ library.  The lunapi Python wrapper contains objects that point to this single reference.  This means that if you copy an instance, both instances point to the same underlying data.  If you change one, you'll see the other is also changed.

We can also access multiple individuals at the same time:

In [ ]:
a = proj.inst(1)
b = proj.inst(2)

In [ ]:
a

In [ ]:
b

Note that `a` and `b` are simply references to the underlying data, which are stored within the core Luna C/C++ library.  

In [ ]:
a = proj.inst(0)
a

In [ ]:
b = a
b

In [ ]:
a.stat()

In [ ]:
b.stat()

In [ ]:
a.eval( 'MASK ifnot=N2 & RE' )
a.stat()

In [ ]:
b.stat()

However... if you create two... can be diff

In [ ]:
p1 = proj.inst(0)
p2 = proj.inst(0)

In [ ]:
p1

In [ ]:
p2

In [ ]:
p1.stat()

In [ ]:
p2.stat()

In [ ]:
p1.eval('MASK ifnot=N2 & RE')

In [ ]:
p1.stat()

In [ ]:
p2.stat()

That is, the unique underlying instances are created by the `inst()` function.  One innvocation of `inst()` == one instance. 